In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import os
import random


device = "cuda" if torch.cuda.is_available() else "cpu"
assert device == "cuda"  

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)


# input [Batch (sentence), L (max line length), V (corpus size)] with probabilities 
# output : [B,2]

In [ ]:
embedding_size = 256
seq_length = 32
print(vocab_size)

embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=0)

# LSTM (RNN) Classifier

In [ ]:
#LSTM Classifier

batch_size = 4
LSTMlayers = 3
vocab_size = len(unique(vocab_counts))


lstm = nn.LSTM(input_size=256, hidden_size=256, num_layers=LSTMlayers, batch_first=True)
dropout = nn.Dropout(0.5)
fc1 =  nn.Linear(in_features=256, out_features=257)
fc2 = nn.Linear(257, 1)

# def initialize_states():
#   initial_hidden_state:
#   initial_memory_state: torch.zeros()

def LSTM_forward(seq):
  # hidden_state = 
  seq = embedding(seq)
  output, (hidden,cell) = lstm(seq)
  output = dropout(output)
  print(output.shape)
  output = fc1(output[:,-1])
  print(output.shape)
  output = F.relu(output)
  output = fc2(output)
  output = F.sigmoid(output)

  return output.reshape(output.size(1), -1)

# CNN Classifier

In [ ]:
filter_size = [2,3,4]
n_filters = [100,100,100]
n_class = 2
fc = nn.Linear(np.sum(n_filters), 1)
dropout = nn.Dropout(0.5)

def CNN_forward(seq):
      seq = embedding(seq)
      seq_reshaped = seq.permute(0,2,1)
      conv_list = []
      pool_list = []
      for convolution1d in nn.ModuleList([nn.Conv1d(256, n_filters[i],filter_size[i]) for i in range(3)]):
        conv_list.append(F.relu(convolution1d(seq_reshaped)))
      
      for conv in conv_list:
        pool_list.append(F.max_pool1d(conv, conv.shape[2]))
      
      concat = []
      for pool in pool_list:
        concat.append(pool.squeeze(dim=2))      
      concat = torch.cat(concat, dim=1)

      output = fc(concat)
      output = dropout(output)
      output = torch.sigmoid(output)

      # return logits
      return output.reshape(output.size(1), -1)
      # output looks like : tensor([[0.9315, 0.5000]], grad_fn=<ViewBackward>)

loss = 0 
def loss_fcn(taylor_batch, drake_batch): 
  target_taylor =torch.tensor([[1,1]])
  target_drake =torch.tensor([[0,0]])   
  target_taylor=target_taylor.type(torch.FloatTensor)
  target_drake=target_drake.type(torch.FloatTensor)

  #define and add optimizer
  taylor_loss = F.binary_cross_entropy(taylor_batch,target_taylor)
  drake_loss = F.binary_cross_entropy(drake_batch,target_drake)
  return taylor_loss, drake_loss

# loss += loss_fcn[0] + loss_fcn[1]